<a href="https://colab.research.google.com/github/hfathie/hfvSPH_on_GPU/blob/main/getDensity_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%%writefile test.cu
#include <iostream>
#include <fstream>
#include <cmath>
#include <string>
#include <vector>
#include <sstream>
#include "myCppSPHLibs.h"
using namespace std;

const int N = 131504;
const float my_pi = 3.141592f;


int main(){

  // Reading Hydra file.
  string fname = "Hydra_130k.csv";

  vector<vector<string>> content;
  vector<string> row;
  string line, word;
  
  fstream file (fname, ios::in);
  if(file.is_open())
  {
  while(getline(file, line))
  {
  row.clear();
  
  stringstream str(line);
  
  while(getline(str, word, ','))
  row.push_back(word);
  content.push_back(row);
  }
  }
  else
  cout<<"Could not open the file\n";

  float *x,*y,*z, *h, *mass, *rho;
  float *d_x,*d_y,*d_z, *d_h, *d_mass, *d_rho;

  x = new float[N];
  y = new float[N];
  z = new float[N];

  rho = new float[N];
  h = new float[N];
  mass = new float[N];

  // 0  1  2  3   4   5    6   7  8  9  10
  // x, y, z, vx, vy, vz, rho, P, c, h, m.

  for(int i=0; i<N; i++){

    x[i] = stof(content[i][0]);
    y[i] = stof(content[i][1]);
    z[i] = stof(content[i][2]);

    rho[i] = stof(content[i][6]);
    h[i] = stof(content[i][9]);
    mass[i] = stof(content[i][10]);
  }

  cudaMalloc(&d_x, N*sizeof(float));
  cudaMalloc(&d_y, N*sizeof(float));
  cudaMalloc(&d_z, N*sizeof(float));

  cudaMalloc(&d_rho, N*sizeof(float));
  cudaMalloc(&d_h, N*sizeof(float));
  cudaMalloc(&d_mass, N*sizeof(float));

  // Copy from Host to Device.
  cudaMemcpy(d_x, x, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_y, y, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_z, z, N*sizeof(float), cudaMemcpyHostToDevice);

  cudaMemcpy(d_rho, rho, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_h, h, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_mass, mass, N*sizeof(float), cudaMemcpyHostToDevice);

  // Launching the kernel on GPU
  int blockSize = 256; // number of threads in a block
  int gridSize = (N + blockSize - 1) / blockSize; // Number of blocks in a grid

  getDensity<<<gridSize, blockSize>>>(d_x, d_y, d_z, d_mass,
                                      d_rho, d_h, my_pi, N);

  // Wait for the GPU to finish before accessing the Host
  cudaDeviceSynchronize();


  // visual inspection
  for(int i = 0; i < 10; i++){
    cout << rho[i] << endl;
  }

  // Free memory.
  cudaFree(d_x);
  cudaFree(d_y);
  cudaFree(d_z);

  cudaFree(d_rho);
  cudaFree(d_h);
  cudaFree(d_mass);

  delete[] x;
  delete[] y;
  delete[] z;

  delete[] rho;
  delete[] h;
  delete[] mass;

}

Overwriting test.cu


In [5]:
%%shell
nvcc test.cu -o test

In [6]:
%%shell
./test

0.35458
0.247541
0.406348
0.148354
0.497729
0.286529
0.279707
0.421166
0.327605
0.247348


In [7]:
%%shell
nvprof ./test

==306== NVPROF is profiling process 306, command: ./test
0.35458
0.247541
0.406348
0.148354
0.497729
0.286529
0.279707
0.421166
0.327605
0.247348
==306== Profiling application: ./test
==306== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.89%  250.98ms         1  250.98ms  250.98ms  250.98ms  getDensity(float*, float*, float*, float*, float*, float*, float, int)
                    0.11%  273.28us         6  45.546us  45.312us  45.887us  [CUDA memcpy HtoD]
      API calls:   54.80%  306.53ms         6  51.088ms  3.0380us  306.40ms  cudaMalloc
                   44.88%  251.02ms         1  251.02ms  251.02ms  251.02ms  cudaDeviceSynchronize
                    0.17%  946.93us         6  157.82us  136.10us  176.49us  cudaMemcpy
                    0.07%  378.02us         1  378.02us  378.02us  378.02us  cuDeviceTotalMem
                    0.05%  262.34us         6  43.723us  2.8410us  141.85us  cudaFree
         